In [10]:
import pandas as pd
import numpy as np
import json

from pathlib import Path

from config import config
from src.preprocessing.merge_csv_in_df import merge_csv_by_date
from src.llm.loaders.gnews_loader import load_and_process_news_data
from src.llm.token_estimator import estimate_tokens_and_cost
from src.llm.langchain_news_analyzer import CryptoNewsSentimentAnalyzer


df_btc = pd.read_csv(
    config.DATA_DIR / "processed" / "crypto_prices" / "btc_original.csv",
    parse_dates=["date"],
    index_col="date",
)
df_eth = pd.read_csv(
    config.DATA_DIR / "processed" / "crypto_prices" / "eth_original.csv",
    parse_dates=["date"],
    index_col="date",
)

# Google Trends


we have google trends searches for the words:

- bitcoin
- blockchain
- cryptocurrency
- ethereum
- investing


In [11]:
GOOGLE_TRENDS = config.DATA_DIR / "raw" / "google_trends"

df_btc = merge_csv_by_date(
    GOOGLE_TRENDS / "bitcoin.csv",
    df_btc,
    "gTrendsBitcoin",
    is_monthly=True,
    rows_to_skip=1,
)
df_btc = merge_csv_by_date(
    GOOGLE_TRENDS / "blockchain.csv",
    df_btc,
    "gTrendsBlockchain",
    is_monthly=True,
    rows_to_skip=1,
)
df_btc = merge_csv_by_date(
    GOOGLE_TRENDS / "cryptocurrency.csv",
    df_btc,
    "gTrendsCryptocurrency",
    is_monthly=True,
    rows_to_skip=1,
)
df_btc = merge_csv_by_date(
    GOOGLE_TRENDS / "investing.csv",
    df_btc,
    "gTrendsInvesting",
    is_monthly=True,
    rows_to_skip=1,
)

df_eth = merge_csv_by_date(
    GOOGLE_TRENDS / "ethereum.csv",
    df_eth,
    "gTrendsEthereum",
    is_monthly=True,
    rows_to_skip=1,
)
df_eth = merge_csv_by_date(
    GOOGLE_TRENDS / "blockchain.csv",
    df_eth,
    "gTrendsBlockchain",
    is_monthly=True,
    rows_to_skip=1,
)
df_eth = merge_csv_by_date(
    GOOGLE_TRENDS / "cryptocurrency.csv",
    df_eth,
    "gTrendsCryptocurrency",
    is_monthly=True,
    rows_to_skip=1,
)
df_eth = merge_csv_by_date(
    GOOGLE_TRENDS / "investing.csv",
    df_eth,
    "gTrendsInvesting",
    is_monthly=True,
    rows_to_skip=1,
)

# Gold


In [12]:
df_btc = merge_csv_by_date(config.DATA_DIR / "raw" / "gold.csv", df_btc, "goldValue")
df_eth = merge_csv_by_date(config.DATA_DIR / "raw" / "gold.csv", df_eth, "goldValue")

# SP 500


In [13]:
df_btc = merge_csv_by_date(config.DATA_DIR / "raw" / "sp500.csv", df_btc, "sp500Value")
df_eth = merge_csv_by_date(config.DATA_DIR / "raw" / "sp500.csv", df_eth, "sp500Value")

In [14]:
pd.set_option("display.max_columns", None)
df_btc.head()

,open,high,low,close,volume,marketCap,logPriceChange,priceMovement,localMin_7,localMax_7,localMin_14,localMax_14,localMin_21,localMax_21,dayOfWeek_Sin,dayOfWeek_Cos,EMA_12,EMA_26,RSI_14,BB_Middle,BB_Upper,BB_Lower,OBV,AO,KAMA,PPO,PPO_Signal,PPO_Histogram,PVO,PVO_Signal,PVO_Histogram,ROC,RSI,Stoch_RSI_K,Stoch_RSI_D,Stoch_K,Stoch_D,TSI,Ultimate_Oscillator,WilliamsR,ADI,CMF,EMV,FI,MFI,NVI,VPT,BBM,BBW,DCM,DCW,KCM,KCW,UI,Aroon_down,Aroon_up,CCI,DPO,Ichimoku_A,Ichimoku_B,Ichimoku_Base,Ichimoku_Conversion,KST,MACD,MACD_Signal,MI,TRIX,Vortex_down,Vortex_up,WMA,CR,PSAR_down,PSAR_up,gTrendsBitcoin,gTrendsBlockchain,gTrendsCryptocurrency,gTrendsInvesting,goldValue,sp500Value
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-01,963.658,1003.08,958.699,998.325,147775008.0,1.605041e+10,0.035254,1,0,0,0,0,0,0,-0.781831,0.623490,927.913019,871.434686,83.084032,871.46615,1027.196258,715.736042,2.922337e+09,139.503888,955.043802,9.281661,6.612502,2.669159,14.687295,17.065823,-2.378528,15.474703,83.084032,56.506121,62.292639,93.416834,94.168225,62.537845,82.395616,-2.234913,2.957941e+09,0.562083,268.189478,2.396025e+09,78.950576,1007.506291,1.475275e+11,974.5448,0.053900,889.9595,226.241,891.830567,107.944512,82.395616,0.0,100.0,111.969673,55.45900,745.59775,704.6545,881.415,932.7520,21126.993631,56.478334,46.010969,27.806359,0.435426,0.523588,1.376167,958.571873,66.168065,0,1,6,16,1,37,NaN,NaN
2017-01-02,998.617,1031.39,996.702,1021.750,222184992.0,1.642902e+10,0.023193,1,0,0,0,0,0,0,0.000000,1.000000,942.349478,882.569153,84.887313,883.52585,1046.419181,720.632519,3.144522e+09,143.060506,965.552321,9.859751,7.261952,2.597799,16.843606,17.021380,-0.177774,10.820795,84.887313,62.800521,59.793982,95.188902,93.928779,63.913117,82.121062,-4.018458,3.056633e+09,0.583475,304.980903,2.797262e+09,80.534253,1007.506291,2.270175e+11,983.7106,0.093940,904.1145,254.551,904.203846,110.878559,82.121062,0.0,100.0,131.923933,82.62590,746.11650,704.6545,898.401,946.9070,22315.365979,59.780324,48.764840,27.991673,0.455892,0.493168,1.426380,972.880582,69.774625,0,1,6,16,1,37,NaN,NaN
2017-01-03,1021.600,1044.08,1021.600,1043.840,185168000.0,1.678637e+10,0.021389,1,0,0,0,0,0,0,0.781831,0.623490,957.963404,894.515142,86.363588,896.64380,1066.653829,726.633771,3.329690e+09,147.687241,989.282139,10.251627,7.859887,2.391740,16.391334,16.895370,-0.504037,16.134229,86.363588,68.222461,62.509701,97.882847,95.496194,65.565071,83.501748,-0.098089,3.237848e+09,0.602965,318.138014,2.981991e+09,81.475127,1009.668268,1.932858e+11,997.7792,0.129162,910.9410,266.278,917.502527,108.882506,83.501748,0.0,100.0,139.181140,108.60445,746.54300,704.6545,904.746,970.4890,23941.222052,63.448262,51.701525,28.176019,0.477714,0.496120,1.463899,989.391727,72.727175,0,1,6,16,1,37,1160.400024,2257.830078
2017-01-04,1044.400,1159.42,1044.400,1154.730,344945984.0,1.857187e+10,0.100960,1,0,1,0,1,0,1,0.974928,-0.222521,988.235188,913.790317,91.076165,915.47590,1110.400038,720.551762,3.674636e+09,167.045691,1049.890184,10.613740,8.410658,2.203083,22.662037,18.048704,4.613333,28.849800,91.076165,79.549401,70.190795,98.147193,97.072980,69.471661,85.562083,-1.441875,3.554663e+09,0.673299,449.937853,8.020429e+09,83.325859,1009.668268,3.983195e+11,1036.4776,0.250110,969.1915,380.457,940.095620,142.543219,85.562083,0.0,100.0,214.597295,223.78505,752.11150,704.6545,962.416,1031.8375,26693.551368,74.444872,56.250194,28.821680,0.504156,0.422958,1.423311,1023.428000,82.152330,0,1,6,16,1,37,1163.800049,2270.750000
2017-01-05,1156.730,1191.10,910.417,1013.380,510199008.0,1.630025e+10,-0.130575,0,0,0,0,0,0,0,0.433884,-0.900969,992.103621,921.167330,61.771598,926.89955,1116.585647,737.213453,3.164437e+09,177.824844,1048.171858,10.545420,8.837610,1.707810,31.635758,20.766115,10.869643,11.653684,61.771598,56.209877,67.993913,81.462850,92.497630,57.008442,64.793651,-54.071487,3.418777e+09,0.535435,192.256444,-3.427722e+09,67.351462,1009.668268,5.170255e+11,1046.4050,0.214551

# Google News


In [15]:
df_news_btc = load_and_process_news_data(
    config.DATA_DIR / "raw" / "news_articles" / "bitcoin_articles.json"
)

df_news_eth = load_and_process_news_data(
    config.DATA_DIR / "raw" / "news_articles" / "ethereum_articles.json"
)

In [16]:
df_news_btc = df_news_btc.reset_index()
df_news_btc = df_news_btc.drop_duplicates(subset=['complete_text'], keep='first')

df_news_eth = df_news_eth.drop_duplicates(subset=['complete_text'], keep='first')
df_news_eth = df_news_eth.reset_index()

df_news_btc.head()

,date,title,subtitle,complete_text,word_count
0,2017-01-01,"A Look At Bitcoin Bubbles, When Will the Next ...",There have been four noteworthy bubbles in bit...,"A Look At Bitcoin Bubbles, When Will the Next ...",35
1,2017-01-01,"Bitcoin Price Tops $1,000 in First Day of 2017...",The price of bitcoin inched upward over the co...,"Bitcoin Price Tops $1,000 in First Day of 2017...",36
2,2017-01-01,What Will the Bitcoin Price Be in 2017?,CoinDesk's Charles Bovaird asks the experts fo...,What Will the Bitcoin Price Be in 2017? CoinDe...,25
3,2017-01-01,"Bitcoin Breaks $1,000 as Exchanges Break Volum...",Global bitcoin exchanges report record-setting...,"Bitcoin Breaks $1,000 as Exchanges Break Volum...",29
4,2017-01-01,"Bitcoin Price Tops $1,000 in First Day of 2017...",The price of bitcoin inched upward over the co...,"Bitcoin Price Tops $1,000 in First Day of 2017...",36


In [17]:
news_btc_cost = estimate_tokens_and_cost(
    df_news_btc["complete_text"].tolist(),
    cost_per_million_tokens=0.15,
    word_in_tokens=1.8,
)
news_eth_cost = estimate_tokens_and_cost(
    df_news_eth["complete_text"].tolist(),
    cost_per_million_tokens=0.15,
    word_in_tokens=1.8,
)

print("BTC News Cost")
print(f"Total Texts Processed: {news_btc_cost['total_texts']}")
print(f"Total Tokens Estimated: {news_btc_cost['total_tokens']}")
print(f"Estimated Cost: ${news_btc_cost['estimated_cost']:.2f} \n")

print("ETH News Cost")
print(f"Total Texts Processed: {news_eth_cost['total_texts']}")
print(f"Total Tokens Estimated: {news_eth_cost['total_tokens']}")
print(f"Estimated Cost: ${news_eth_cost['estimated_cost']:.2f}")

BTC News Cost
Total Texts Processed: 20741
Total Tokens Estimated: 1221579
Estimated Cost: $0.18 

ETH News Cost
Total Texts Processed: 16937
Total Tokens Estimated: 984198
Estimated Cost: $0.15


In [18]:
analyzer_btc = CryptoNewsSentimentAnalyzer(verbose=True, output_file_path=config.DATA_DIR / "temp" / "news_sentiment_btc.csv")
result_df_btc = analyzer_btc.analyze_articles_in_range(
    df_news_btc, start_date="2017-10-19", end_date="2022-12-31"
)

Processed article 2398 | Date: 2017-10-18 | Score: 2.0
Processed article 2399 | Date: 2017-10-18 | Score: 4.0
Processed article 2400 | Date: 2017-10-18 | Score: 7.0
Processed article 2401 | Date: 2017-10-18 | Score: 8.0


KeyboardInterrupt: 

In [ ]:
analyzer_eth = CryptoNewsSentimentAnalyzer(
    verbose=True, output_file_path=config.DATA_DIR / "temp" / "news_sentiment_eth.csv"
)
result_df_eth = analyzer_eth.analyze_articles_in_range(
    df_news_eth, start_date="2017-01-01", end_date="2022-12-31"
)